In [4]:
import geopandas as gpd
import rasterio as rio
from rasterio.plot import show
import numpy as np
import scipy.special
import gdal
import matplotlib.cm
import folium
from folium import plugins
from math import sqrt

**Auxiliary functions**

In [140]:
#Get bouding box of tiff image

def GetExtent(ds):
    geo_t = ds.GetGeoTransform()
    x_size, y_size = ds.RasterXSize, ds.RasterYSize
    xmin = min(geo_t[0], geo_t[0] + x_size * geo_t[1])
    xmax = max(geo_t[0], geo_t[0] + x_size * geo_t[1])
    ymin = min(geo_t[3], geo_t[3] + y_size * geo_t[5])
    ymax = max(geo_t[3], geo_t[3] + y_size * geo_t[5])
    return xmin, xmax, ymin, ymax

In [142]:
#Convert tiff to numpy matrix with bounding box

def convert_raster_to_mappable(path_raster):
  #Open raster file
  driver=gdal.GetDriverByName('GTiff')
  driver.Register()

  ds = gdal.Open(path_raster) 
  if ds is None:
      print('Could not open')

  #Get coordinates, cols and rows
  cols = ds.RasterXSize 
  rows = ds.RasterYSize 

  xmin, xmax, ymin, ymax = GetExtent(ds)

  #Raster convert to array in numpy
  bands = ds.RasterCount
  band=ds.GetRasterBand(1)
  dataset= band.ReadAsArray(0,0,cols,rows)

  #Normalize colors
  dataset = scipy.special.expit(dataset)

  return dataset, xmin, xmax, ymin, ymax

In [7]:
#Convert tiff to gdal bands, bounding box and coordinates bands

def convert_esa_raster_to_mappable(path_raster):
  #Open raster file
  driver=gdal.GetDriverByName('GTiff')
  driver.Register()

  ds = gdal.Open(path_raster) 
  if ds is None:
      print('Could not open')

  #Get coordinates, cols and rows
  cols = ds.RasterXSize 
  rows = ds.RasterYSize 

  band_lat=ds.GetRasterBand(1)
  dataset_lat=band_lat.ReadAsArray(0,0,cols,rows)*1e-6
  band_lon=ds.GetRasterBand(2)
  dataset_lon= band_lon.ReadAsArray(0,0,cols,rows)*1e-6

  xmin, xmax, ymin, ymax = dataset_lat.min(), dataset_lat.max(), dataset_lon.min(), dataset_lon.max()

  return ds, xmin, xmax, ymin, ymax, cols, rows, dataset_lat, dataset_lon

In [8]:
#color map for folium

def colormap(colors, x):
  result = [0,0,0,x]
  for i in range(3):
    if colors[i] != 0:
      result[i] = colors[i]
  return result

In [87]:
#rotate satellite data according to coordinates bands

def reframe_image(input_image, input_coordx, input_coordy, xmin, xmax, ymin, ymax, cols, rows):
  step = sqrt((input_coordx[0,1] - input_coordx[0,0])**2 + (input_coordy[0,1] - input_coordy[0,0])**2)
  n = round((xmax-xmin)/step) + 1
  m = round((ymax-ymin)/step) + 1
  output_image = np.zeros((m,n))

  for i in range(rows):
    for j in range(cols):

      output_image[round((input_coordy[i,j]-ymin)/step),round((input_coordx[i,j]-xmin)/step)] = input_image[i,j]
  
  return output_image

In [18]:
#add image to folium map

def add_image_to_map(m, dataimage, xmin, xmax, ymin, ymax, title, colormap):
  
  #add data
  img = folium.raster_layers.ImageOverlay(
          name=title,
          image=dataimage,
          bounds=[[xmin, ymin], [xmax, ymax]],
          opacity=1,
          interactive=True,
          cross_origin=False,
          zindex=1,
          colormap=colormap,
      )

  img.add_to(m)

**Mapping**

In [147]:
#initialize map
m = folium.Map([21, 10], zoom_start=4)


In [148]:
#https://geodata.lib.berkeley.edu/catalog/stanford-gp836bd8268
shapefile = 'asset/stanford-gp836bd8268-geojson.json'
gdf = gpd.read_file(shapefile)

#add district boundaries
folium.GeoJson(gdf, name="borders").add_to(m)

In [ ]:
#https://data.humdata.org/dataset/mauritania-roads
style = {'fillColor': '#FF8C00', 'color': '#FF8C00'}

shapefile = 'asset/mrt_rds_1m_ons.json'
gdf = gpd.read_file(shapefile)

#add district roads
folium.GeoJson(gdf, name="roads").add_to(m)

In [ ]:
#https://data.humdata.org/dataset/hotosm_mrt_railways
style = {'fillColor': '#228B22', 'color': '#228B22'}

shapefile = 'asset/hotosm_mrt_railways_lines.json'
gdf = gpd.read_file(shapefile)

#add district railways
folium.GeoJson(gdf, name="railway lines", style_function=lambda x:style).add_to(m)

In [98]:
#https://scihub.copernicus.eu/dhus/odata/v1/Products('dc25ee6b-a0df-4fcf-bed0-60568adf4389')/$value
#with http://step.esa.int/main/download/snap-download/ to get to .tiff

channel_name = ["IWV", "OGVI", "OTCI"]
ds, xmin, xmax, ymin, ymax, cols, rows, input_coordx, input_coordy= convert_esa_raster_to_mappable('asset/subset_S3B_OL_2_LFR____20210421T103548_20210421T103848_20210422T155447_0179_051_279_2520_LN1_O_NT_002.tif')

#add top satellite image
#/!\ it is costly
for channel in range(3):
  band=ds.GetRasterBand(channel + 2)
  dataimage=band.ReadAsArray(0,0,cols,rows)
  #dataimage=scipy.special.expit(dataimage)
  dataimage = reframe_image(dataimage, input_coordx, input_coordy, xmin, xmax, ymin, ymax, cols, rows)
  add_image_to_map(m, dataimage, xmin, xmax, ymin, ymax, channel_name[channel] + " top", lambda x: colormap(np.eye(3)[channel],x))


In [97]:
#https://scihub.copernicus.eu/dhus/odata/v1/Products('dc25ee6b-a0df-4fcf-bed0-60568adf4389')/$value
#with http://step.esa.int/main/download/snap-download/ to get to .tiff

channel_name = ["IWV", "OGVI", "OTCI"]
ds, xmin, xmax, ymin, ymax, cols, rows, input_coordx, input_coordy = convert_esa_raster_to_mappable('asset/subset_S3B_OL_2_LFR____20210421T103848_20210421T104148_20210422T155510_0179_051_279_2700_LN1_O_NT_002.tif')

#add bottom satellite image
#/!\ it is costly
for channel in range(3):
  band=ds.GetRasterBand(channel + 2)
  dataimage=band.ReadAsArray(0,0,cols,rows)
  #dataimage=scipy.special.expit(dataimage)
  dataimage = reframe_image(dataimage, input_coordx, input_coordy, xmin, xmax, ymin, ymax, cols, rows)
  add_image_to_map(m, dataimage, xmin, xmax, ymin, ymax, channel_name[channel] + " bottom", lambda x: colormap(np.eye(3)[channel],x))


In [149]:
#ftp://ftp.worldpop.org/GIS/Population_Density/Global_2000_2020_1km/2020/MRT/mrt_ppp_2015_1km_Aggregated.tif
dataimage, xmin, xmax, ymin, ymax = convert_raster_to_mappable('asset/mrt_ppp_2015_1km_Aggregated.tif')

#add population
add_image_to_map(m, dataimage, ymin, ymax, xmin, xmax, "population 2015", lambda x: (1, 0, 1, x))

In [150]:
#ftp://ftp.worldpop.org/GIS/Births/Individual_countries/MRT/MRT_pregs_pp_v2_2015.tif
dataimage, xmin, xmax, ymin, ymax = convert_raster_to_mappable('asset/MRT_pregs_pp_v2_2015.tif')

#add births
add_image_to_map(m, dataimage, ymin, ymax, xmin, xmax, "births 2015", lambda x: (1, 1, 0, x))

In [151]:
#add layers
folium.LayerControl().add_to(m)

#show
m